In [1]:
# do basic imports and unpack McMurdo data

from pmagpy import ipmag
reload(ipmag)
from pmagpy import pmag
from programs import new_builder as nb
from programs import data_model3
reload(data_model3)
import os
import pandas as pd
import numpy as np
from pandas import DataFrame
from programs.new_builder import Contribution

import pmagpy.controlled_vocabularies3 as cv



/Users/nebula/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


got full_df
got parsed_df
got full_df
got parsed_df


In [2]:
dir_path = os.path.join(os.getcwd(), '3_0', 'Megiddo')
con = Contribution(dir_path)

loc_dm = con.tables['locations'].data_model.dm['locations']
loc_df = con.tables['locations'].df
site_dm = con.tables['sites'].data_model.dm['sites']
site_df = con.tables['sites'].df
samp_df = con.tables['samples'].df
samp_dm = con.tables['samples'].data_model.dm['samples']
spec_df = con.tables['specimens'].df
spec_dm = con.tables['specimens'].data_model.dm['specimens']
age_df = con.tables['ages'].df
age_dm = con.tables['ages'].data_model.dm['ages']
meas_df = con.tables['measurements'].df
meas_dm = con.tables['measurements'].data_model.dm['measurements']
cont_df = con.tables['contribution'].df
cont_dm = con.tables['contribution'].data_model.dm['contribution']
crit_df = con.tables['criteria'].df
crit_dm = con.tables['criteria'].data_model.dm['criteria']


current_con = con
current_df = loc_df
current_dm = loc_dm
# replace NaN with None
current_dm = current_dm.where((pd.notnull(current_dm)), None)
cols = current_df.columns


#vocab.get_controlled_vocabularies()

# mess up some validations for locations
current_df.loc['Tel Hazor']['lat_s'] = 400.
current_df['dir_inc'] = 5
current_con.tables.pop('sites')

# mess up some validations for sites
#current_df.pop('age')
#current_df['dir_tilt_correction'] = 1
#current_df['dir_tilt_correction'] = 'one'

# mess up some validations for samples
#current_df.pop('citations')
#current_df.iloc[0].lon = 600.
#current_df.iloc[1].lat = 'hello'

# mess up some validations for measurements
#current_df.loc['mgh05a01:LP-PI-TRM1', 'magn_moment'] = 2
#current_df.loc['mgh05a01:LP-PI-TRM1', 'specimen'] = "fake_specimen"
#current_df.pop('experiment')

current_df.head()

-W- No such file: /Users/nebula/Python/PmagPy/3_0/Megiddo/images.txt


,citations,lat_n,lat_s,location,location_type,lon_e,lon_w,dir_inc
location,,,,,,,,
Tel Hazor,This study,33.0168,400,Tel Hazor,Archeological Site,35.568,35.568,5
Tel Megiddo,This study,32.585,32.585,Tel Megiddo,Archeological Site,35.185,35.185,5


In [3]:
import pmagpy.controlled_vocabularies3 as cv
reload(cv)
vocab = cv.Vocabulary()
vocabulary, possible_vocabulary = vocab.get_controlled_vocabularies()



-I- Importing controlled vocabularies from https://earthref.org


In [4]:
# check column validity
invalid_cols = [col for col in cols if col not in current_dm.index]


# need to add requiredOneInGroup

def requiredUnless(col_name, df, arg, *args):
    arg_list = arg.split(",")
    arg_list = [arg.strip('"') for arg in arg_list]
    msg = ""
    for a in arg_list:
        if "." in a:
            continue
        if a not in df.columns:
            msg += "{} is required unless {} is present.  ".format(col_name, a)
    if msg:
        return msg
    else:
        return None
    return None


def requiredUnlessTable(col_name, df, arg, *args):
    """
    Col_name must be present in df unless
    arg (table_name) is present in contribution
    """
    table_name = arg
    if col_name in df.columns:
        return None
    elif table_name in current_con.tables:
        return None
    else:
        #print "{} is required unless table {} is present".format(col_name, table_name)
        return "{} is required unless table {} is present".format(col_name, table_name)

    
def requiredIfGroup(col_name, df, arg, *args):
    """
    Col_name is required if other columns of 
    the group arg are present.
    """
    group_name = arg
    groups = set()
    columns = df.columns
    for col in columns:
        if col not in current_dm.index:
            continue
        group = current_dm.loc[col]['group']
        groups.add(group)
    if group_name in groups:
        if col_name in columns:
            return None
        else:
            #print "{} is required if column group {} is used".format(col_name, group_name)
            return "{} is required if column group {} is used".format(col_name, group_name)
    return None


def required(col_name, df, arg):
    #print df
    if col_name in df.columns:
        return None
    else:
        return "{} is required".format(col_name) 
  
def is_in(row, col_name, arg, *args):
    cell_value = row[col_name]
    if "." in arg:
        table_name, table_col_name = arg.split(".")
        if table_name not in current_con.tables:
            return "Must contain a value from {} table. Missing {} table.".format(table_name, table_name)
        possible_values = current_con.tables[table_name].df[table_col_name].unique()
        if cell_value not in possible_values:
            return "This value: {} is not found in: {}".format(cell_value, arg)
    return None
    
def check_max(row, col_name, arg, *args):
    #print 'check_max'
    cell_value = row[col_name]
    if not cell_value:
        return None
    try:
        arg = float(arg)
    except ValueError:
        arg = row[arg]
    arg = float(arg)
    try:
        if float(cell_value) <= float(arg):
            return None
        else:
            return "{} must be <= {}".format(str(cell_value), str(arg))
    # this happens when the value isn't a float (an error which will be caught elsewhere)
    except ValueError:
        return None

def check_min(row, col_name, arg, *args):
    cell_value = row[col_name]
    if not cell_value:
        return None
    try:
        arg = float(arg)
    except ValueError:
        arg = row[arg]
    try:
        if float(cell_value) >= float(arg):
            return None
        else:
            return "{} must be >= {}".format(str(cell_value), str(arg))
    # this happens when the value isn't a float (an error which will be caught elsewhere)
    except ValueError:
        return None

def cv(row, col_name, arg, current_data_model):
    cell_value = row[col_name]
    if not cell_value:
        return None
    if cell_value.lower() in [v.lower() for v in vocabulary[col_name]]:
        return None
    else:
        return "{} is not in {}".format(cell_value, arg)
        

# validate presence
presence_operations = {"required": required, "requiredUnless": requiredUnless,
                       "requiredIfGroup": requiredIfGroup, 
                       'requiredUnlessTable': requiredUnlessTable}
# validate values
value_operations = {"max": check_max, "min": check_min, "cv": cv, "in": is_in}

def split_func(string):
    """
    Take a string like 'requiredIf("arg_name")'
    return the function name and the argument:
    (requiredIf, arg_name)
    """
    ind = string.index("(")
    return string[:ind], string[ind+1:-1].strip('"')


def test_type(value, value_type):
    if not value:
        return None
    if value_type == "String":
        if str(value) == value:
            return None
        else:
            return "should be string"
    elif value_type == "Number":
        try:
            float(value)
            return None
        except ValueError:
            return "should be a number"
    elif value_type == "Integer":
        if isinstance(value, str):
            if str(int(value)) == value:
                return None
            else:
                return "should be an integer"
        else:
            if int(value) == value:
                return None
            else:
                return "should be an integer"
    else:
        return None
    #String, Number, Integer, List, Matrix, Dictionary, Text
    


def validate_df(df, dm):
    for validation_name, validation in dm.iterrows():
        value_type = validation['type']
        #print 'value_type', value_type
        if validation_name in df.columns:
            output = df[validation_name].apply(test_type, args=(value_type,))
            df["type_" + validation_name] = output

        val_list = validation['validations']
        if not val_list:
            continue
        for num, val in enumerate(val_list):
            func_name, arg = split_func(val)
            # first validate for presence
            if func_name in presence_operations:
                func = presence_operations[func_name]
                grade = func(validation_name, current_df, arg)
                pass_col_name = "presence_pass_" + validation_name + "_" + func.__name__
                df[pass_col_name] = grade
    
            # then validate for correct values
            elif func_name in value_operations:
                func = value_operations[func_name]
                if validation_name in df.columns:
                    grade = df.apply(func, args=(validation_name, arg, dm), axis=1)
                    df["pass_" + validation_name + "_" + func.__name__] = grade.astype(object)
    return df

                
current_df = validate_df(current_df, current_dm)

    
# check that values pass validation
# validation checks to add:
# sv (suggested vocab)
# requiredOneInGroup


# re-do upload_magic to use contribution-level (??)

# first, do validations on each table in the contribution
# this will include removing unneeded data (RmKeys from old upload_magic)
# this will also include checking everything against the data model (strings are strings, etc.)g

# next, do cross-contribution validations (all specimens exist, etc.)

# next, splat out each table into a file and wrap it up.  give it a sensible name.  

pass_cols = current_df.columns.str.match("^pass_")
present_cols = current_df.columns.str.match("^presen")
type_cols = current_df.columns.str.match("^type_")
val_cols = np.where([bool(v) for v in pass_cols], [bool(v) for v in pass_cols], [bool(v) for v in present_cols])

present_col_names = current_df.columns[present_cols]
current_df[present_col_names].dropna(how='all', axis=1).head()

current_df[current_df.columns[type_cols]].dropna(how='all', axis=1).head()



""
location
Tel Hazor
Tel Megiddo


In [5]:
current_df[current_df.columns[present_cols]].dropna(how='all', axis=1).head()


,presence_pass_age_requiredUnless,presence_pass_age_high_requiredUnless,presence_pass_age_low_requiredUnless,presence_pass_age_unit_required,presence_pass_dir_dec_requiredIfGroup,presence_pass_dir_tilt_correction_requiredIfGroup,presence_pass_geologic_classes_required,presence_pass_lithologies_required,presence_pass_method_codes_requiredUnlessTable,presence_pass_result_quality_requiredUnlessTable,presence_pass_result_type_requiredUnlessTable
location,,,,,,,,,,,
Tel Hazor,age is required unless age_low is present. ag...,age_high is required unless age is present.,age_low is required unless age is present.,age_unit is required,dir_dec is required if column group Direction ...,dir_tilt_correction is required if column grou...,geologic_classes is required,lithologies is required,method_codes is required unless table sites is...,result_quality is required unless table sites ...,result_type is required unless table sites is ...
Tel Megiddo,age is required unless age_low is present. ag...,age_high is required unless age is present.,age_low is required unless age is present.,age_unit is required,dir_dec is required if column group Direction ...,dir_tilt_correction is required if column grou...,geologic_classes is required,lithologies is required,method_codes is required unless table sites is...,result_quality is required unless table sites ...,result_type is required unless table sites is ...


In [6]:
current_df[current_df.columns[pass_cols]].dropna(how='all', axis=1).head()


,pass_lat_n_check_min,pass_lat_s_check_max
location,,
Tel Hazor,33.0168 must be >= 400.0,400.0 must be <= 33.0168
Tel Megiddo,None,None


In [7]:
current_df[current_df.columns[type_cols]].dropna(how='all', axis=1).head()

""
location
Tel Hazor
Tel Megiddo


## Filling in an existing dataframe

In [8]:
# keep all of df1, add in any extra from df2
df1 = pd.DataFrame(np.random.randint(1, 10, (3, 5)), columns=['one', 'two', 'three', 'four', 'five'])
df1.iloc[0, 1] = np.nan
df1.iloc[2, 2] = np.nan
df2 = pd.DataFrame(np.random.randint(1, 10, (3, 5)), columns=['one', 'three', 'five', 'seven', 'nine'])
df1

,one,two,three,four,five
0,4,NaN,1.0,9,1
1,9,8.0,6.0,7,9
2,6,4.0,NaN,2,4


In [9]:
df2

,one,three,five,seven,nine
0,5,4,1,7,2
1,7,9,8,9,1
2,1,7,7,5,9


In [10]:
unique_df2_cols = df2.columns.difference(df1.columns)
unique_df2 = df2[unique_df2_cols]

# this adds in all the unique columns that weren't in df1
concat_df = pd.concat([df1, unique_df2], axis=1)
# fills in null values in df1 with values from df2
concat_df.fillna(df2)

,one,two,three,four,five,nine,seven
0,4,NaN,1.0,9,1,2,7
1,9,8.0,6.0,7,9,1,9
2,6,4.0,7.0,2,4,9,5
